In [40]:
from sympy import (
    solve,
    Poly,
    Eq,
    Function,
    exp,
    Indexed,
    IndexedBase,
    Tuple,
    sqrt,
    Symbol,
    sin,
    limit,
    root,
    real_root,
    S,
    nroots,
)
from sympy.solvers.solvers import unrad
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.optimize import minimize
from collections import namedtuple


In [41]:
rate_range = 41  # $/hr rate range 150 - 350 by 5s
hour_range = 401  # hours index from 0.0 - 100.0 by 0.25s
cols = [150 + (iter * 5) for iter in range(rate_range)]
index = [iter * 0.25 for iter in range(hour_range)]


In [42]:
df = pd.DataFrame((np.outer(index, cols))) # init df, label
df = df.set_axis([cols], axis=1)
df = df.set_axis([index], axis=0)


In [43]:
budget = float(input("budget")) # test query with value present in all columns

In [44]:
mask = df.mask(df != budget, False) # replace in place with 'False' anything != budget 

In [45]:
mask = mask[mask.any(axis=1)] # mask matching value, dropping all rows/columns that don't contain hot values
mask_id = mask.loc[:, (mask != 0).any(axis=0)]


In [46]:
mask#test

,150,155,160,165,170,175,180,185,190,195,...,305,310,315,320,325,330,335,340,345,350
10.00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12.00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12.50,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15.00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18.75,False,False,3000.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20.00,3000.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [47]:
mask[mask > 0].stack().index.tolist() # Render key/index list of values == budget. Everything works and I am incredible.

[(10.0, 300), (12.0, 250), (12.5, 240), (15.0, 200), (18.75, 160), (20.0, 150)]

In [48]:
len(mask) # experiment with len as tool in defs

6

In [49]:
mask_id.iloc[0].keys() # keys of mask

MultiIndex([(150,),
            (160,),
            (200,),
            (240,),
            (250,),
            (300,)],
           )

In [50]:
m = mask[mask > 0].stack().index.tolist() # experiment with subtracting mask[0] from budget. I am not the smartest person alive.
df0 = pd.DataFrame([budget]).values.tolist()
df1 = pd.DataFrame(m[0]).values.tolist()

In [51]:
float_df1_0 = np.float64(df1[0])
float_df1_1 = np.float64(df1[1])

pe = float_df1_0 * float_df1_1
print(pe)


[3000.]


In [ ]:
from bisect import bisect_left # Plan from here is to loop this against 'budget' and then subtract the return from budget, then repeat with return.


def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return after
    else:
        return before


In [104]:
def is_valid(x):
    if len(mask) > 0:
        return True
    else:
        return False


In [105]:
def breakdown(x):
    while True:
        if is_valid(x):
            i = (yield x)
            if i is not None:
                x = i
        x += 1

In [107]:
breakdown(3000)

<generator object breakdown at 0x152e95310>